# Interactive Life Expectancy Simulator

Adjust health predictors to see how the predicted life expectancy changes and view the distribution of the predictor you just changed.

Use the dropdown to select a 2020 country to auto-populate sliders with its values.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, HTML

# Load data
coefs_df = pd.read_csv('health_unscaled_coefficients.csv')
data_df  = pd.read_csv('health_predictors.csv')
sns.set_theme(style='whitegrid')

In [2]:
# Extract intercept
intercept = coefs_df.loc[coefs_df['variable']=='Intercept', 'beta_unscaled'].iat[0]
# Build slopes dict for all other variables
slopes = (
    coefs_df[coefs_df['variable']!='Intercept']
    .set_index('variable')['beta_unscaled']
    .to_dict()
)
# Exclude unwanted variable if present
slopes.pop('Lifetime risk of maternal death (1 in: rate varies by country)', None)
predictors = list(slopes.keys())
predictors

['Population ages 0-14, total',
 'People practicing open defecation (% of population)',
 'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
 'Domestic private health expenditure per capita (current US$)',
 'Prevalence of overweight (modeled estimate, % of children under 5)',
 'Number of maternal deaths',
 'Lifetime risk of maternal death (%)',
 'Incidence of tuberculosis (per 100,000 people)',
 'Prevalence of undernourishment (% of population)']

In [3]:
def predict_le(vals):
    '''Compute predicted life expectancy given a dict of predictor values.'''
    return intercept + sum(slopes[var] * vals[var] for var in predictors)

In [4]:
# Country dropdown for 2020
countries_2020 = sorted(data_df.loc[data_df['Year']==2020, 'CountryShortName'].unique())
country_dd = widgets.Dropdown(
    options=countries_2020,
    description='Use 2020 Country:',
    layout=widgets.Layout(width='400px')
)

# Create sliders
sliders = {}
for var in predictors:
    col = data_df[var].dropna()
    min_val, max_val = float(col.min()), float(col.max())
    mu = coefs_df.loc[coefs_df['variable']==var, 'mean'].iat[0]
    step = (max_val - min_val) / 100.0
    sliders[var] = widgets.FloatSlider(
        value=mu,
        min=min_val,
        max=max_val,
        step=step,
        description=var,
        continuous_update=False,
        layout=widgets.Layout(width='1200px'),
        style={'description_width': '600px'}
    )

# Display all controls
ui = widgets.VBox([country_dd] + list(sliders.values()))
display(ui)

In [5]:
out = widgets.Output()
display(out)

def update(var):
    out.clear_output()
    vals = {v: sliders[v].value for v in predictors}
    pred = predict_le(vals)
    with out:
        display(HTML(f"<h2 style='color:darkblue;'>Predicted Life Expectancy: {pred:.2f} years</h2>"))
        plt.figure(figsize=(6,4))
        sns.histplot(data_df[var].dropna(), bins=30, kde=True)
        plt.axvline(sliders[var].value, color='red', linestyle='--')
        plt.title(f"Distribution of {var}")
        plt.xlabel(var)
        plt.tight_layout()
        plt.show()

def on_country_change(change):
    country = change['new']
    row = data_df[(data_df['Year']==2020) & (data_df['CountryShortName']==country)].iloc[0]
    for v in predictors:
        sliders[v].value = float(row[v])
    # refresh using the first predictor
    update(predictors[0])

# Attach observers
for w in sliders.values():
    w.unobserve_all()
for var, w in sliders.items():
    w.observe(lambda change, var=var: update(var), names='value')
country_dd.observe(on_country_change, names='value')

# initial draw
update(predictors[0])

Output()